# Empty

In [ ]:
when you 

This snippet disables scrolling.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) { return false; }

In [ ]:
import xarray as xr

import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.data as qndata


def load_data(period):
    return qndata.futures_load_data(tail=period)


def strategy(data):
    close = data.sel(field='close')
    sma200 = qnta.sma(close, 200).isel(time=-1)
    sma20 = qnta.sma(close, 20).isel(time=-1)
    return xr.where(sma200 < sma20, 1, 0)


qnbt.backtest(
    competition_type="futures",
    load_data=load_data,
    lookback_period=365,
    test_period=15*365,
    strategy=strategy
)

In [ ]:
import xarray as xr
import numpy as np

import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.data as qndata


def load_data(period):
    futures = qndata.futures_load_data(tail=period)
    crypto = qndata.crypto_load_data(tail=period)
    return { "futures": futures, "crypto": crypto }, futures.time.values


def window(data, max_date: np.datetime64, lookback_period:int):
    min_date = max_date - np.timedelta64(lookback_period,'D')
    return {
        "futures":data['futures'].sel(time=slice(min_date,max_date)),
        "crypto":data['crypto'].sel(time=slice(min_date,max_date)),
    }


def strategy(data):
    close = data['futures'].sel(field='close')
    sma200 = qnta.sma(close, 200).isel(time=-1)
    sma20 = qnta.sma(close, 20).isel(time=-1)
    return xr.where(sma200 < sma20, 1, 0)


qnbt.backtest(
    competition_type="futures",
    load_data=load_data,
    lookback_period=365,
    test_period=2*365,
    strategy=strategy,
    window=window
)